In [1]:
%matplotlib inline
import cv2
from pylab import *
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.transforms as transforms
# from torchsummary import summary
from dataset import CelebA

from torch.autograd import Variable

import torch.optim as optim
import sys

from albumentations import (
    HorizontalFlip,
    VerticalFlip,
    Normalize,
    Compose,
    PadIfNeeded,
    RandomCrop,
    Rotate,
    Resize
)
from models import LinkNet34
# from fastai.vision import *
# from fastai.callbacks.hooks import *
# from fastai.utils.mem import *
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class LossBinary:

    def __init__(self, jaccard_weight=0):
        self.nll_loss = nn.BCEWithLogitsLoss()
        self.jaccard_weight = jaccard_weight

    def __call__(self, outputs, targets):
        loss = (1 - self.jaccard_weight) * self.nll_loss(outputs, targets)

        if self.jaccard_weight:
            eps = 1e-15
            jaccard_target = (targets == 1).float()
            jaccard_output = F.sigmoid(outputs)

            intersection = (jaccard_output * jaccard_target).sum()
            union = jaccard_output.sum() + jaccard_target.sum()

            loss -= self.jaccard_weight * torch.log((intersection + eps) / (union - intersection + eps))
        return loss

In [ ]:
root = '/media/nasir/Drive1/datasets/celeba_real/'
size = 256
train_aug= Compose([
            HorizontalFlip(p=0.5),
            Rotate(15),
         ])
trainset = CelebA(root = root, train=True, augmentation=train_aug)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=4)


valset = CelebA(root = root, train=False)

valset_loader = torch.utils.data.DataLoader(valset, batch_size=4,
                                          shuffle=True, num_workers=4)


In [ ]:
def freeze(model, l):
    total_params = 0
    for i, param in enumerate(model.parameters()):
        total_params+=1
    for i, param in enumerate(model.parameters()):
        if i < total_params - l:
            param.requires_grad = False
        else:
            param.requires_grad = True
            
    return model

In [ ]:
valid_loss_es = []
def validate(net, criterion):
    net.eval()
    valid_loss = 0.0
    for i, (imgs, true_masks) in enumerate(valset_loader):
        imgs, true_masks = Variable(imgs.cuda()), Variable(true_masks.to(dtype=torch.float, device = device))


        masks_pred = net(imgs)
        masks_probs_flat = masks_pred.view(-1)
        true_masks_flat = true_masks.view(-1)

        loss = criterion(masks_probs_flat, true_masks_flat)
        valid_loss += loss.item()
        
    net.train()
    valid_loss = valid_loss/i
    
    valid_loss_es.append(valid_loss)
    best_valid_loss = min(valid_loss_es)

    print(f'validation loss {round(valid_loss, 4)}, {round(best_valid_loss, 4)}')
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        print('saving ... ')
        torch.save(net.state_dict(), 'linknet.pth')

In [ ]:
def train(model, trainloader, epochs, lr=1e-2):
    optimizer = optim.SGD(model.parameters(),
        lr=lr,
        momentum=0.9,
        weight_decay=0.0005
    )
    cosine = optim.lr_scheduler.CosineAnnealingLR(optimizer, 2)
    criterion = LossBinary(jaccard_weight=1)
#     criterion = nn.BCELoss()
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for i, (imgs, true_masks) in enumerate(trainloader):

            imgs, true_masks = Variable(imgs.cuda()), Variable(true_masks.to(dtype=torch.float, device = device))


            optimizer.zero_grad()
            masks_pred = model(imgs)
            masks_probs_flat = masks_pred.view(-1)
            true_masks_flat = true_masks.view(-1)
            loss = criterion(masks_probs_flat, true_masks_flat)
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()
#             if i % 10 == 9:
            sys.stdout.write(f'\rEpoch: {epoch} ---- Loss: {round(epoch_loss/(i+1), 4)}')
            sys.stdout.flush()
            cosine.step()
            
        sys.stdout.write('\n')
        validate(model, criterion)
        sys.stdout.write('\n')

In [ ]:
model = LinkNet34()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.train()
model.to(device)
model.load_state_dict(torch.load('linknet.pth'))

summary(model, (3, 256, 256))


In [ ]:
train(model, trainloader, 5, lr=1e-2)

In [ ]:
train(model, trainloader, 5, lr=1e-3)

In [ ]:
train(model, trainloader, 10, lr=1e-4)